In [ ]:
import os, glob
import os.path

from google.cloud import storage

bucket = storage.Client().get_bucket('bigsister-butcket')
for blob in bucket.list_blobs():
  print(blob.name)

bucket = storage.Client().get_bucket('bigsister-butcket')
for blob in bucket.list_blobs():


In [8]:
import zipfile
from google.cloud import storage

bucket = storage.Client().get_bucket('cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')

with zipfile.ZipFile('https://storage.googleapis.com/cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/inside_croki_02.zip', 'r') as zip_ref:
    zip_ref.extractall('https://storage.googleapis.com/cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')

FileNotFoundError: [Errno 2] No such file or directory: 'https://storage.googleapis.com/cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/inside_croki_02.zip'

In [9]:
import shutil
shutil.unpack_archive('https://storage.googleapis.com/cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/inside_croki_02.zip', 'https://storage.googleapis.com/cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')

ReadError: https://storage.googleapis.com/cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/inside_croki_02.zip is not a zip file

In [1]:
from google.cloud import storage
from zipfile import ZipFile
from zipfile import is_zipfile
import io

def zipextract(bucketname, zipfilename_with_path):

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketname)

    destination_blob_pathname = zipfilename_with_path

    blob = bucket.blob(destination_blob_pathname)
    zipbytes = io.BytesIO(blob.download_as_string())

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, 'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                blob = bucket.blob(zipfilename_with_path + "/" + contentfilename)
                blob.upload_from_string(contentfile)

#zipextract("cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66", "inside_croki_02.zip") # if the file is gs://mybucket/path/file.zip

In [2]:
bucket = storage.Client().get_bucket('cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')
ziplist = []

for blob in bucket.list_blobs():
    if str(blob.name).endswith('zip'):
        ziplist.append(blob.name)
ziplist

['Assult/inside_croki_02.zip',
 'Assult/inside_croki_03.zip',
 'Assult/insidedoor_02.zip',
 'Assult/insidedoor_03.zip',
 'Assult/outsidedoor_02.zip',
 'Assult/outsidedoor_03.zip',
 'Datefight/inside_croki_02.zip',
 'Datefight/inside_croki_03.zip',
 'Datefight/insidedoor_02.zip',
 'Datefight/insidedoor_03.zip',
 'Datefight/outsidedoor_03.zip',
 'Datefight/outsidedoor_04.zip',
 'Fight/inside_croki_02.zip',
 'Fight/inside_croki_03.zip',
 'Fight/insidedoor_02.zip',
 'Fight/insidedoor_03.zip',
 'Fight/outsidedoor_02.zip',
 'Fight/outsidedoor_03.zip']

In [3]:
for z in ziplist:
    if not z.startswith('As'):
        zipextract("cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66", z)
        print(z)

Datefight/inside_croki_02.zip
Datefight/inside_croki_03.zip
Datefight/insidedoor_02.zip
Datefight/insidedoor_03.zip
Datefight/outsidedoor_03.zip
Datefight/outsidedoor_04.zip
Fight/inside_croki_02.zip
Fight/inside_croki_03.zip
Fight/insidedoor_02.zip
Fight/insidedoor_03.zip
Fight/outsidedoor_02.zip
Fight/outsidedoor_03.zip


In [ ]:
!pip install cloudstorage

In [22]:
dir(gcs)

['Batch',
 'Blob',
 'Bucket',
 'Client',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_helpers',
 '_http',
 '_signing',
 'acl',
 'batch',
 'blob',
 'bucket',
 'client',
 'constants',
 'fileio',
 'hmac_key',
 'notification',
 'retry',
 'version']

In [ ]:
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket('cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')

In [42]:
# 동영상의 전체 시간, 행동의 레이블, 시작 시간, 종료 시간 추출
from google.cloud import storage
from xml.etree.ElementTree import parse
import xml.etree.ElementTree as ET
import logging
import os

bucket = client.get_bucket('cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')

# 이상행동일떄
def parsefile(path):
    client = storage.Client()
    blob = bucket.get_blob(path)
    xml_file = blob.download_as_string()
  # tree = parse(path)
  # root = tree.getroot()
    root = ET.fromstring(xml_file)
    print(root)

    event = root.findall("event")
    header = root.findall("header")

    endtime = [x.findtext("duration") for x in header]
    starttime = [x.findtext("starttime") for x in event]
    duration = [x.findtext("duration") for x in event]
    event = [x.findtext("eventname") for x in event][0]

    #event = [x.findtext("event") for x in student]
    endlist = endtime[0][3:-2].split(':')
    startlist = starttime[0][3:-2].split(':') # [분, 초] : 시작 시간의 분, 초
    durationlist = duration[0][3:-2].split(':') # [분, 초] : 행동 지속시간의 분, 초

    alltime = int(endlist[0])*60 + int(endlist[1])
    starttime = int(startlist[0])*60 + int(startlist[1]) # 이상 행동 시작 시간 (초)
    end_action_time = starttime + int(durationlist[0])*60 + int(durationlist[1]) # 전체 행동이 끝나는 시간 = getduration... ^^
    return f'gs://cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/{path[:-4]}.mp4, 0, {alltime}, {event}, {starttime}, {end_action_time}'

In [43]:
parsefile('Assult/inside_croki_03.zip/407-3/407-3_cam01_assault01_place04_day_spring.xml')

<Element 'annotation' at 0x7f6adde83950>


'gs://cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/Assult/inside_croki_03.zip/407-3/407-3_cam01_assault01_place04_day_spring.mp4, 0, 311, assault, 63, 289'

In [ ]:
from google.cloud import storage
bucket = storage.Client().get_bucket('cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66')
for blob in bucket.list_blobs():
    print(blob.name)

In [45]:
def make_csv_file(path):
    f = open("visionai.csv", 'w')
    f.write('VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END\n')
    for blob in bucket.list_blobs():
        if str(blob.name).endswith('xml'):
            csvline = parsefile(blob.name)+'\n'
            f.write(csvline)
    f.close()

make_csv_file('gs://cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66/')

<Element 'annotation' at 0x7f6adde7c3b0>
<Element 'annotation' at 0x7f6adde835f0>
<Element 'annotation' at 0x7f6adde7cfb0>
<Element 'annotation' at 0x7f6adde834d0>
<Element 'annotation' at 0x7f6addf22e30>
<Element 'annotation' at 0x7f6addd4ce90>
<Element 'annotation' at 0x7f6addf22ef0>
<Element 'annotation' at 0x7f6addd4cbf0>
<Element 'annotation' at 0x7f6adde7c1d0>
<Element 'annotation' at 0x7f6addd4c950>
<Element 'annotation' at 0x7f6adde81e30>
<Element 'annotation' at 0x7f6addd4c650>
<Element 'annotation' at 0x7f6adde81e30>
<Element 'annotation' at 0x7f6addd4c770>
<Element 'annotation' at 0x7f6adde81d70>
<Element 'annotation' at 0x7f6addf22650>
<Element 'annotation' at 0x7f6addeb1110>
<Element 'annotation' at 0x7f6adde81f50>
<Element 'annotation' at 0x7f6addfbd110>
<Element 'annotation' at 0x7f6adde81e90>
<Element 'annotation' at 0x7f6addfbd3b0>
<Element 'annotation' at 0x7f6adde81f50>
<Element 'annotation' at 0x7f6addf22890>
<Element 'annotation' at 0x7f6adde81e90>
<Element 'annota

IndexError: list index out of range

In [25]:
# Instantiates a client
client = storage.Client()

bucket_name = 'gs://cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66'

bucket = client.get_bucket(bucket_name)

blob = bucket.get_blob('Assult/inside_croki_02.zip/24-5/24-5_cam01_assault01_place09_night_spring.xml')

downloaded_blob = blob.download_as_text()

print(downloaded_blob)

NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/gs://cloud-ai-platform-4a6abf55-7af9-4a88-b2ce-7e8551bbfc66?projection=noAcl&prettyPrint=false: Not Found